# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

#### План работы
###### Шаг 1. Откроем, изучим и подготовим данные
###### Шаг 2. Обучим и проверим модель для каждого региона
    Разобьём данные на обучающую и валидационную выборки в соотношении 75:25
    Обучим модель и сделаем предсказания на валидационной выборке
    Сохраним предсказания и правильные ответы на валидационной выборке
    Покажем средний запас предсказанного сырья и RMSE модели
    Проанализируем результаты
###### Шаг 3. Подготовка к расчёту прибыли
    Все ключевые значения для расчётов сохраним в отдельных переменных
    Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины. Сравним полученный объём сырья со средним запасом в каждом регионе
    Напишем выводы по этапу подготовки расчёта прибыли
###### Шаг 4. Напишем функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели
    Выберем скважины с максимальными значениями предсказаний
    Просуммируем целевое значение объёма сырья, соответствующее этим предсказаниям
    Рассчитаем прибыль для полученного объёма сырья
###### Шаг 5. Посчитаем риски и прибыль для каждого региона
    Применим технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли
    Найдём среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль
    Напишем выводы: предложим регион для разработки скважин и обоснуем выбор

### Описание данных

    Данные геологоразведки трёх регионов:

    id — уникальный идентификатор скважины;
    f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
    product — объём запасов в скважине (тыс. баррелей).

#### Условия задачи:

    Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
    При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
    Бюджет на разработку скважин в регионе — 10 млрд рублей.
    При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
    После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

## Загрузка и подготовка данных

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats as st
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [ ]:
data_0 = pd.read_csv('/datasets/geo_data_0.csv')
data_1 = pd.read_csv('/datasets/geo_data_1.csv')
data_2 = pd.read_csv('/datasets/geo_data_2.csv')

In [ ]:
data_0.shape

(100000, 5)

In [ ]:
data_1.shape

(100000, 5)

In [ ]:
data_2.shape

(100000, 5)

In [ ]:
data_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [ ]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [ ]:
data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


Пропусков в данных нет.

In [ ]:
data_0.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


In [ ]:
data_1.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


In [ ]:
data_2.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


id не несет нужной для расчетов информации, удаляем.

In [ ]:
data_0 = data_0.drop(['id'], axis=1)
data_1 = data_1.drop(['id'], axis=1)
data_2 = data_2.drop(['id'], axis=1)

#### Вывод
    импортировали все необходимые инструменты
    загрузили датасеты и ознакомились с ними
    выяснили, что пропусков в данных нет
    удалили столбец id
    нулевые значения заменить на среднее нельзя и удалять тоже не стоит.

## Обучение и проверка модели

In [ ]:
model = LinearRegression()

Фунцией разобьем данные на выборки, обучим модель и сделаем предсказания

In [ ]:
def get_predictions_target(data):
    features = data.drop(['product'], axis = 1)
    target = data['product']
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=.25, random_state=666)
    model.fit(features_train, target_train)
    return pd.Series(model.predict(features_valid)), target_valid

Сохраним предсказания и правильные ответы.

In [ ]:
predicted_0, target_0 = get_predictions_target(data_0)
predicted_1, target_1 = get_predictions_target(data_1)
predicted_2, target_2 = get_predictions_target(data_2)

Средний запас предсказанного сырья для каждого региона и RMSE

In [ ]:
def get_product_mean_rmse (answers, predictions):
    mse = mean_squared_error(answers, predictions)
    print('Средний запас, тысяч баррелей:', predictions.mean())
    print('RMSE:', mse ** 0.5)

In [ ]:
print('Месторождение 1')
get_product_mean_rmse(target_0, predicted_0)

Месторождение 1
Средний запас, тысяч баррелей: 92.59883747361133
RMSE: 37.78852914975086


In [ ]:
print('Месторождение 2')
get_product_mean_rmse(target_1, predicted_1)

Месторождение 2
Средний запас, тысяч баррелей: 69.119604542144
RMSE: 0.8899276270922485


In [ ]:
print('Месторождение 3')
get_product_mean_rmse(target_2, predicted_2)

Месторождение 3
Средний запас, тысяч баррелей: 95.05831290258574
RMSE: 40.008583723061925


#### Вывод:

    разбили данные на обучающую и валидационную выборки
    обучили модели и сделали предсказания, сохранили их и правильные ответы
    посчитали средний запас сырья для каждого региона по предсказаниям и RMSE
    качественнее всего модель отработала для региона №2, здесь метрика RMSE с самым маленьким значением
    средний предсказанный запас сырья больше всего у регионов 1 и 3, больше 90 тыс. бареллей. Здесь же значение RMSE получилось очень большим

## Подготовка к расчёту прибыли

Данные для расчета прибыли сохраним в переменные:

In [ ]:
COUNT_OF_WELLS = 200

Бюджет на разработку скважин в регионе — 10 млрд рублей.

In [ ]:
BUDGET = 10000000000

При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.

In [ ]:
REVENUE_1K_BAR = 450000

Бюджет на одну скважину

In [ ]:
BUDGET_FOR_ONE = BUDGET / COUNT_OF_WELLS

максимальная доля убытков

In [ ]:
MAX_RISK = 0.025

Объём сырья для безубыточной разработки новой скважины.

In [ ]:
BUDGET_FOR_ONE / REVENUE_1K_BAR

111.11111111111111

In [ ]:
print("Средний запас сырья в регионах, тыс. баррелей:")
print()
print("Регион 1:", target_0.mean())
print("Регион 2:", target_1.mean())
print("Регион 3:", target_2.mean())

Средний запас сырья в регионах, тыс. баррелей:

Регион 1: 92.7335845019874
Регион 2: 69.12651161786542
Регион 3: 95.29994745917732


#### Вывод
    111.111 тыс. баррелей в одной скважине - минимальный объем сырья, для безубыточной разработки.
    Если сравнивать со средним объемом, который мы посчитали в каждом регионе, то проект будет убыточным в любом регионе. 

Напишем функцию рассчета прибыли по 200 скважинам каждого региона, из которых получается самый большой объем сырья.

In [ ]:
def profit(target, probabilities):
    target = pd.Series(target).reset_index(drop=True)
    probabilities = pd.Series(probabilities).reset_index(drop=True)
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:COUNT_OF_WELLS]
    mlrd = 1000000000
    return ((REVENUE_1K_BAR * selected.sum()) - BUDGET) / mlrd

In [ ]:
profit(target_0, predicted_0)

3.3651062961070406

In [ ]:
profit(target_1, predicted_1)

2.4150866966815108

In [ ]:
profit(target_2, predicted_2)

2.0007895216939278

Cамым прибыльным является регион 1 - выручка здесь больше 3-х млрд рублей.

## Расчёт прибыли и рисков 

Посчитаtем риски и прибыль для каждого региона

In [ ]:
state = np.random.RandomState(12345)

In [ ]:
def bootstrap (target, predictions):
    values = []
    for i in range(1000):
        target_subsample = target.reset_index(drop=True).sample(n=500, replace=True, random_state=state)
        pred_subsample = predictions[target_subsample.index]
        rev = profit(target_subsample, pred_subsample)
        values.append(rev)

    values = pd.Series(values)
    lower = values.quantile(MAX_RISK)
    #confidence_interval = st.t.interval(0.95, len(values)-1, values.mean(), values.sem())
    confidence_interval = (values.quantile(0.025), values.quantile(0.975))
    mean = values.mean()

    print("Средняя выручка:", mean)
    print("2.5%-квантиль:", lower)
    print("Доверительный интервал:", confidence_interval)
    print("Риски убытка:", (values < 0).mean())

In [ ]:
bootstrap(target_0, predicted_0)

Средняя выручка: 0.43069839930828546
2.5%-квантиль: -0.10671743695636005
Доверительный интервал: (-0.10671743695636005, 0.9307130825630421)
Риски убытка: 0.052


In [ ]:
bootstrap(target_1, predicted_1)

Средняя выручка: 0.48536834632742604
2.5%-квантиль: 0.08284952787924067
Доверительный интервал: (0.08284952787924067, 0.8787523728795791)
Риски убытка: 0.005


In [ ]:
bootstrap(target_2, predicted_2)

Средняя выручка: 0.3484111749716312
2.5%-квантиль: -0.18284438789284505
Доверительный интервал: (-0.18284438789284505, 0.8567012626397866)
Риски убытка: 0.098


#### Вывод
Исходя из проведенного анализа, можно сказать, что 'Месторождение 2' самое выгодное, об этом говорит и показатель RMSE и процент убытков.
